In [1]:
# Import Libraries
import numpy as np
import pandas as pd

from collections import defaultdict
import threading
import time

# Database Integration Libraries
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import datetime

In [2]:
# !pip install SQLAlchemy 

In [3]:
# # A pre-defined menu of food and beverages
# menu = {
#     1: {"name": "Pizza", "price": 10.99},
#     2: {"name": "Burger", "price": 8.99},
#     3: {"name": "Pasta", "price": 12.99},
#     4: {"name": "Salad", "price": 6.99},
#     5: {"name": "Soda", "price": 1.99},
#     6: {"name": "Coffee", "price": 2.99},
# }

In [4]:
# # A Dict to store orders by food item:
# order_queue = defaultdict(list)

In [5]:
# # Function to dispatch orders after 15 seconds
# def dispatch_orders(food_item):
#     print(f"\nDispatching order for {food_item}:")
#     for order in order_queue[food_item]:
#         print(f" - Deliver to Table {order['table']} (ordered at {time.ctime(order['timestamp'])})")
#     # Clear the orders for the food item after dispatch
#     order_queue[food_item].clear()

# # Function to queue order and set 15-second timer for dispatch
# def queue_order(food_item, table):
#     current_time = time.time()
#     order = {"table": table, "timestamp": current_time}
#     # Add the order to the queue for the specified food item
#     order_queue[food_item].append(order)
#     # If this is the first order for this item, start a 15-second timer to dispatch
#     if len(order_queue[food_item]) == 1:
#         print(f"\nNew order received for {food_item}. Starting a 15-second countdown.")
#         threading.Timer(0.25 * 60, dispatch_orders, [food_item]).start()  # 15-second countdown
#     else:
#         print(f"Similar order for {food_item} added to existing batch.")

# # Function to display the menu
# def display_menu():
#     print("\n--- Menu ---")
#     for item_id, details in menu.items():
#         print(f"{item_id}. {details['name']} - ${details['price']:.2f}")

# # Function to take an order from the customer
# def take_order():
#     display_menu()
#     try:
#         item_number = int(input("\nEnter the number of the food item you want to order: "))
#         if item_number not in menu:
#             print("Invalid selection. Please choose a valid item from the menu.")
#             return
#         table = input("Enter Table number: ")
#         food_item = menu[item_number]["name"]
#         queue_order(food_item, table)
#         print(f"Order for {food_item} at table {table} has been placed.")
#     except ValueError:
#         print("Please enter a valid number.")

# # User-driven interaction for managing orders
# def simulation():
#     while True:
#         print("\nChoose an action:")
#         print("1. Place an Order")
#         print("2. View Active Orders")
#         print("3. Exit Simulation")

#         choice = input("\nEnter choice (1-3): ")

#         if choice == '1':
#             take_order()
#         elif choice == '2':
#             view_active_orders()
#         elif choice == '3':
#             print("\nExiting simulation.")
#             break
#         else:
#             print("\nInvalid choice. Please try again.")

# # Function to view all active orders
# def view_active_orders():
#     if not order_queue:
#         print("\nNo active orders.")
#     else:
#         for food_item, orders in order_queue.items():
#             print(f"\n{food_item} Orders:")
#             for order in orders:
#                 print(f" - Table {order['table']}, ordered at {time.ctime(order['timestamp'])}")


In [6]:
# # Run the improved simulation
# simulation()

In [7]:
# Create and initialize the database
def initialize_database():
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    
    # Create a table for menu items
    cursor.execute('''CREATE TABLE IF NOT EXISTS menu (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        food_item TEXT NOT NULL
                    )''')
    
    # Create a table for orders
    cursor.execute('''CREATE TABLE IF NOT EXISTS orders (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        food_item TEXT NOT NULL,
                        table_number INTEGER NOT NULL,
                        order_time REAL NOT NULL,
                        status TEXT NOT NULL
                    )''')
    
    conn.commit()
    conn.close()

In [8]:
# Function to add a menu item
def add_menu_item(food_item):
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO menu (food_item) VALUES (?)', (food_item,))
    conn.commit()
    conn.close()

# Function to place an order
def place_order(food_item, table_number):
    current_time = time.time()
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO orders (food_item, table_number, order_time, status) VALUES (?, ?, ?, ?)', 
                   (food_item, table_number, current_time, 'pending'))
    conn.commit()
    conn.close()
    
    # Start a 15-second timer for the order if it's the first of its kind
    if not check_pending_orders(food_item):
        threading.Timer(15, dispatch_orders, [food_item]).start()

# Check if there are pending orders for a specific food item
def check_pending_orders(food_item):
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM orders WHERE food_item = ? AND status = ?', (food_item, 'pending'))
    result = cursor.fetchall()
    conn.close()
    return len(result) > 1

# Dispatch the orders
def dispatch_orders(food_item):
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    
    # Select all pending orders for the food item
    cursor.execute('SELECT * FROM orders WHERE food_item = ? AND status = ?', (food_item, 'pending'))
    orders = cursor.fetchall()
    
    if orders:
        print(f"Dispatching order for {food_item}:")
        for order in orders:
            order_id, _, table_number, order_time, _ = order
            print(f" - deliver to Table {table_number} (ordered at {time.ctime(order_time)})")
            # Mark the order as dispatched
            cursor.execute('UPDATE orders SET status = ? WHERE id = ?', ('dispatched', order_id))
    
    conn.commit()
    conn.close()

# Take an order from the customer
def take_order():
    food_item = input("Enter Food Item: ")
    table_number = input("Enter Table number: ")
    
    # Ensure food_item exists in menu before placing an order
    if validate_menu_item(food_item):
        place_order(food_item, table_number)
        print(f"Order for {food_item} at table {table_number} has been placed.")
    else:
        print(f"Invalid menu item: {food_item}. Please select an item from the menu.")

# Validate if the menu item exists
def validate_menu_item(food_item):
    conn = sqlite3.connect('dishdispatch.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM menu WHERE food_item = ?', (food_item,))
    result = cursor.fetchone()
    conn.close()
    return result is not None

# Add some menu items
def setup_menu():
    add_menu_item("Pizza")
    add_menu_item("Burger")
    add_menu_item("Pasta")
    add_menu_item("Salad")
    add_menu_item("Soda")

# Main function to initialize and simulate orders
def main():
    # Initialize the database and setup the menu
    initialize_database()
    setup_menu()
    
    # Simulate taking orders
    while True:
        take_order()


In [9]:
if __name__ == "__main__":
    main()